In [7]:

# =============================================================================
# Village Income 2019
# =============================================================================

import numpy as np
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
os.chdir('C:/Users/rodri/Dropbox/PhD thesis/python')
from data_functions_albert import remove_outliers, gini
os.chdir('C:/Users/rodri/Dropbox/Chied_Field_June_19/Data/Income')
pd.options.display.float_format = '{:,.2f}'.format



# =============================================================================
# Import data
# =============================================================================

data21 = pd.read_stata("income_230719.dta", convert_categoricals=False)
data29 = pd.read_stata("income_290719.dta", convert_categoricals=False)
datasets = [ data21, data29]
data = pd.concat(datasets, sort=False)

## Look at duplicates:
duplicates = pd.value_counts(data['hhid'])


### Cannot find duration: Do it manually
data.loc[data['hhid']==216, 'duration']
data.drop(14, inplace=True)
data.loc[data['hhid']==183, 'duration']
## hh83 (interviewed twice) remove last one
data.drop(data.loc[data['key']=="uuid:36e75912-7236-4bcd-8524-75eed7a0ae9b"].index, inplace=True)
data.loc[data['intervieweename']=='Estery Yson', 'hhid'] = 602
data.loc[data['intervieweename']=='A chiwa Yahaya', 'hhid'] = 8


### Check that no duplicates: 
duplicates = pd.value_counts(data['hhid'])
data.reset_index(drop=True, inplace=True)

## Remove networks questions
x0 = data.columns.get_loc("a1")
x1 = data.columns.get_loc("ownlandyes")
data.drop(data.iloc[:, x0:x1], inplace=True, axis=1)


percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]

list_crops = ['maize', 'groundnut', 'groundbean', 'sweetpotatoe', 'fingermillet', 'sorghum', 'pearlmillet', 'soyabean', 'pigeonpeas', 'cotton', 'nkhwani', 'cassava',  'sugarcane',  'tomatoes',  'therereokra', 'tanaposi' ] 

# Rename some variables
data.rename(columns={'unitssoldpearlmillet2':'unitssoldpearlmilletout2'}, inplace=True) 
data.rename(columns={'unitssoldsoyabean2':'unitssoldsoyabeanout2'}, inplace=True)
data.rename(columns={'soldquantitygroundbeanin':'soldquantitygroundbeanin'}, inplace=True)


## Remove 9999 observations=====================================
data.replace([9999, 9999.00], np.nan, inplace=True)


# =============================================================================
# Check number, size, and value plots
# =============================================================================
data['total_plots'] = data['manyplot'].fillna(0) + data['rentinmany'].fillna(0)
sumplots = data[['manyplot','rentinmany','total_plots']].describe(percentiles=percentiles)
N_plots = int(data[['total_plots']].max())


### Change Augustine corrections
data.loc[data['hhid']==44, 'areaplot_1' ] = 1
data.loc[data['hhid']==2, 'areaplot_1' ] = 0.25
data.loc[data['hhid']==135, 'manyplot'] = 3




## Check plot size and value =========================================
#units area plots
units_plot = pd.value_counts(data['unitsareaplot_1'])
# small futbol fields are around 1 acre
# square meters to acres: 0.000247105

for i in range(1,N_plots+1):
    data['area_plot_acr_'+str(i)] = data['areaplot_'+str(i)]
    data.loc[data['unitsareaplot_'+str(i)]==2.0, 'area_plot_acr_'+str(i)] = data.loc[data['unitsareaplot_'+str(i)]==2.0, 'areaplot_'+str(i)]*2.47105
    data.loc[data['unitsareaplot_'+str(i)]==4.0, 'area_plot_acr_'+str(i)] = data.loc[data['unitsareaplot_'+str(i)]==4.0, 'areaplot_'+str(i)]*0.000247105


## correction Augustine
## Need to remove the questions on each plot for hh 135...
for i in range(1,N_plots+1):
    data.loc[data['hhid']==135,['area_plot_acr_'+str(i),'valueplot_'+str(i) , 'rentoutplot_'+str(i) ]] = np.nan

# area plots
data.loc[ data['hhid']==101,'area_plot_acr_1'] = 1
data.loc[ data['hhid']==101,'area_plot_acr_2'] = 2.5
data.loc[ data['hhid']==113,'area_plot_acr_2'] = 0.5
# value plots
data.loc[ data['hhid']==28,'rentoutplot_1'] = 10000
data.loc[ data['hhid']==101,'rentoutplot_1'] = 8000
data.loc[ data['hhid']==101,'rentoutplot_2'] = 25000
data.loc[ data['hhid']==113,'area_plot_acr_2'] = 0.5
data.loc[ data['hhid']==113,'rentoutplot_2'] = 5000

## corrections Augustine
#plot1
data.loc[data['hhid']==268,'valueplot_1'] = data.loc[ data['hhid']==268,'valueplot_1']/10
data.loc[data['hhid']==115,'rentoutplot_1'] = data.loc[ data['hhid']==115,'rentoutplot_1']*20
data.loc[data['hhid']==31,'rentoutplot_1'] = data.loc[ data['hhid']==31,'rentoutplot_1']*3  #15000
data.loc[data['hhid']==31,'valueplot_1'] = data.loc[ data['hhid']==31,'valueplot_1']/10  #15000
data.loc[data['hhid']==166, 'rentoutplot_1'] = 10000 
data.loc[data['hhid']==166, 'rentoutplot_1'] = 90000

#plot 2
data.loc[data['hhid']==123, 'rentoutplot_2'] = 10000 
data.loc[data['hhid']==268, 'rentoutplot_2'] = 40000
data.loc[data['hhid']==268, 'valueplot_2'] = 400000
data.loc[data['hhid']==43, 'rentoutplot_2'] = 24000
data.loc[data['hhid']==118, 'rentoutplot_2'] = 10000
#plot 3
data.loc[data['hhid']==268, 'areaplot_3'] = 1.5
data.loc[data['hhid']==268, 'valueplot_3'] = 150000



for i in range(1,N_plots+1):
    data['ratio_value_rent_'+str(i)] = np.nan
    data['p_acre_plot_'+str(i)]= np.nan

#Check ratio value vs rentout: 
for i in range(1,N_plots+1):
    data['ratio_value_rent_'+str(i)]  =  data['valueplot_'+str(i)] / data['rentoutplot_'+str(i)] 

## Check price per acre:
for i in range(1,N_plots+1):
    data['p_acre_plot_'+str(i)]  =  data['valueplot_'+str(i)] / data['area_plot_acr_'+str(i)] 
    
# hh aggregate variables
data['hh_area_plots'] = 0
data['hh_rentout_plots'] = 0
data['hh_value_plots'] = 0


### Add at household level:    
for i in range(1,N_plots+1):
    data['hh_area_plots'] += data['area_plot_acr_'+str(i)].fillna(0)
    data['hh_rentout_plots'] += data['rentoutplot_'+str(i)].fillna(0)
    data['hh_value_plots'] += data['valueplot_'+str(i)].fillna(0)

## hh135 we know have 3 plots, total 2.5 acres. 
data.loc[data['hhid']==135, 'hh_area_plots'] = 2.5

data['hh_p_acre_plots']  =  data['hh_value_plots'] / data['hh_area_plots'] 
data['hh_rent_per_acre']  =  data['hh_rentout_plots'] / data['hh_area_plots'] 
data['hh_ratio_value_rent'] = data['hh_value_plots'] / data['hh_rentout_plots'] 



# =============================================================================
# Check: land area, rentout value, and land value at household level
# =============================================================================
sum_hhplots = data[['hh_area_plots','hh_rentout_plots','hh_value_plots', 'hh_ratio_value_rent', 'hh_p_acre_plots']].describe(percentiles=percentiles)
print('')
print('===============================================================')
print('Check: Distribution land at household level')
print('===============================================================')
sum_hhplots



Check: Distribution land at household level


,hh_area_plots,hh_rentout_plots,hh_value_plots,hh_ratio_value_rent,hh_p_acre_plots
count,255.00,255.00,255.00,228.00,247.00
mean,2.67,"22,135.37","312,262.74",inf,"130,225.72"
std,2.01,"22,278.89","508,034.56",nan,"223,409.83"
min,0.00,0.00,0.00,0.00,0.00
5%,0.47,0.00,0.00,0.19,0.00
10%,1.00,0.00,0.00,3.33,0.00
25%,1.40,"10,000.00","75,000.00",6.78,"49,285.71"
50%,2.40,"16,000.00","170,000.00",10.60,"97,402.60"
75%,3.50,"27,000.00","350,000.00",18.78,"143,888.89"
80%,3.99,"30,100.00","430,000.00",20.00,"166,666.67"


In [9]:
#%% =============================================================================
#  Capital
# =============================================================================

data['k_farm'] = 0
for i in range(1,15):
    data['k_farm'] += data['sellfrmeqp_'+str(i)].fillna(0)
    
sum_capital = data['k_farm'].describe()    
print('======= Summary Capital =========')    
sum_capital

======= Summary Capital =========


count      255.00
mean     5,845.41
std      6,673.67
min          0.00
25%      2,300.00
50%      4,500.00
75%      6,950.00
max     60,000.00
Name: k_farm, dtype: float64

In [10]:
# =============================================================================
#%% Convert agricultural outputs to kgs
# =============================================================================

# Import conversion rates
crop_unit = pd.read_csv("crop_conversions_kg.csv")
crop_unit.set_index('unit', inplace=True)

#Generate empty variables
for crop in list_crops:
    data['total_kg_'+crop] = np.nan 
    data['sold_kg_'+crop] = np.nan 
    data['sold_insiders_kg_'+crop] = np.nan 
    data['sold_outsiders_kg_'+crop] = np.nan 
    data['sold_outside_kg_'+crop] = np.nan 
    data['store_kg_'+crop] = np.nan 
    data['lost_kg_'+crop] = np.nan
    data['total2_kg_'+crop] =np.nan
    data['sold_bigger_total_'+crop] = 0
    data['lost_bigger_total_'+crop] = 0
    data['store_bigger_total_'+crop] = 0
    data['soldloststore_bigger_total_'+crop] = 0
    data['p_'+crop] = np.nan
    data['y_'+crop] = 0
    data['y_agric'] = 0
    data[['unitstotal'+crop, 'unitssold'+crop, 'unitsstore'+crop,'unitslost'+crop]].replace(np.nan, 0, inplace=True)

        
# =============================================================================
# Main Loop: Conversion to kgs for all crops and questions 


### change guys that reported other units and there wasnt question other units appearing
data.loc[ data['hhid']==148, 'unitstotalmaize'] = 2
data.loc[ data['hhid']==178, 'unitstotalgroundnut'] = 11
data.loc[ data['hhid']==115, 'unitstotalgroundbean'] = 11
data.loc[ data['hhid']==182, 'unitstotalsorghum'] = 2
data.loc[ data['hhid']==38, 'unitstotalsugarcane'] = 1
data.loc[ data['hhid']==2, 'unitstotalnkhwani'] = 11
data.loc[ data['hhid']==24, ['totalamounttherereokra','unitstotaltherereokra']] = [10,1]
data.loc[ data['hhid']==24, ['totalamounttomatoes','unitstotaltomatoes']] = [20,1]
data.loc[ data['hhid']==38, ['totalamounttomatoes','unitstotaltomatoes']] = [0.5,1]


data.replace(np.nan, 0, inplace=True)
for i in range(len(data)): 
    for crop in list_crops:
        data.iloc[i, data.columns.get_loc('total_kg_'+crop)] = data.iloc[i,data.columns.get_loc('totalamount'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitstotal'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_insiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'in')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'in')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outsiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outside_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out2')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out2')]),'conversionkg']        
        data.iloc[i, data.columns.get_loc('store_kg_'+crop)] = data.iloc[i,data.columns.get_loc('store'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitsstore'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('lost_kg_'+crop)] = data.iloc[i,data.columns.get_loc('lost'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitslost'+crop)]),'conversionkg']
        

for crop in list_crops:
    data['total2_kg_'+crop] = data['sold_kg_'+crop].fillna(0) +data['store_kg_'+crop].fillna(0) +data['lost_kg_'+crop].fillna(0) 
   
### Corrections Augustine
#Maize
data.loc[data['hhid']==239, 'areaplot_1'] = 0.5
data.loc[data['hhid']==239, 'areaplot_2'] = 0.25
data.loc[data['hhid']==239, 'total_kg_maize'] = 150
data.loc[data['hhid']==86, 'total_kg_maize'] = 21*50
#Groundnuts
data.loc[data['hhid']==268, 'total_kg_groundnut'] = 3*50


#Summary total output kg:
sum_kg = (data[['total_kg_maize', 'total_kg_groundnut', 'total_kg_groundbean', 'total_kg_sweetpotatoe', 'total_kg_fingermillet', 'total_kg_sorghum', 'total_kg_pearlmillet', 'total_kg_soyabean', 'total_kg_pigeonpeas', 'total_kg_cotton', 'total_kg_nkhwani', 'total_kg_cassava',  'total_kg_sugarcane',  'total_kg_tomatoes',  'total_kg_therereokra', 'total_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)

print('===============================================================')
print('Check: Distribution of crop production (in kg)')
print('===============================================================')  
sum_kg.dropna(axis=1, how='any')

Check: Distribution of crop production (in kg)


,total_kg_maize,total_kg_groundnut,total_kg_groundbean,total_kg_sweetpotatoe,total_kg_fingermillet,total_kg_sorghum,total_kg_pigeonpeas,total_kg_cotton,total_kg_nkhwani,total_kg_tomatoes,total_kg_therereokra,total_kg_tanaposi
count,237.00,156.00,17.00,18.00,4.00,6.00,103.00,3.00,29.00,8.00,20.00,2.00
mean,337.05,166.96,32.79,145.83,18.12,24.58,45.31,89.67,42.50,19.25,19.25,100.00
std,425.20,144.68,29.42,164.96,21.54,21.00,43.40,91.22,40.94,23.05,19.47,113.14
min,5.00,5.00,5.00,5.00,2.50,2.50,1.00,25.00,5.00,0.50,5.00,20.00
5%,50.00,20.00,5.00,17.75,3.62,3.12,5.00,27.50,5.00,0.68,5.00,28.00
10%,100.00,25.00,5.00,23.50,4.75,3.75,6.00,30.00,9.00,0.85,5.00,36.00
25%,150.00,50.00,12.50,81.25,8.12,8.75,20.00,37.50,25.00,4.00,5.00,60.00
50%,250.00,150.00,25.00,100.00,10.00,20.00,25.00,50.00,40.00,8.75,10.00,100.00
75%,400.00,250.00,50.00,150.00,20.00,42.50,50.00,122.00,50.00,27.50,21.25,140.00
80%,450.00,250.00,50.00,180.00,26.00,50.00,73.00,136.40,50.00,38.00,28.00,148.00


In [11]:

# =============================================================================
#  get selling PRICES per kg
# =============================================================================
for crop in list_crops:
    data['p_'+crop] = (data['soldvalue'+crop].replace(0,np.nan)).dropna()  / (data['sold_kg_'+crop].replace(0,np.nan)).dropna() 
    #DF = data[['soldvalue'+crop, 'sold_kg_'+crop]].dropna()
sum_prices = data[['p_maize', 'p_groundnut', 'p_groundbean', 'p_sweetpotatoe', 'p_fingermillet', 'p_sorghum', 'p_pearlmillet', 'p_soyabean', 'p_pigeonpeas', 'p_cotton', 'p_nkhwani', 'p_cassava',  'p_sugarcane',  'p_tomatoes',  'p_therereokra', 'p_tanaposi']].describe()
print('') 
print('===============================================================')
print('Check: Distribution of prices')
print('===============================================================')  
sum_prices.dropna(axis=1)



Check: Distribution of prices


,p_maize,p_groundnut,p_sweetpotatoe,p_pigeonpeas,p_cotton,p_tomatoes
count,17.00,55.00,5.00,49.00,3.00,3.00
mean,119.63,118.78,96.00,267.75,224.79,233.33
std,36.42,28.84,28.81,222.88,54.20,230.94
min,77.78,45.00,50.00,20.00,162.37,100.00
25%,100.00,100.00,90.00,200.00,207.19,100.00
50%,100.00,120.00,100.00,200.00,252.00,100.00
75%,140.00,128.29,120.00,264.00,256.00,300.00
max,200.00,200.00,120.00,"1,200.00",260.00,500.00


In [12]:

#Get monetary value:
for crop in list_crops:
    data['y_'+crop] =  np.nanmedian(data['p_'+crop])*data['total_kg_'+crop].fillna(0)
    data['y_agric'] += data['y_'+crop].fillna(0)

    
sum_y = data[['y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'y_sweetpotatoe', 'y_nkhwani']].describe(percentiles=percentiles)
print('') 
print('===============================================================')
print('Agricultural Output (rainy season) in MWK')
print('===============================================================')
sum_y


Agricultural Output (rainy season) in MWK


C:\Users\rodri\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


,y_agric,y_maize,y_groundnut,y_pigeonpeas,y_sweetpotatoe,y_nkhwani
count,255.00,255.00,255.00,255.00,255.00,0.00
mean,"49,169.89","31,325.49","12,256.47","3,660.00","1,029.41",nan
std,"56,946.15","41,888.33","16,722.71","7,077.96","5,676.32",nan
min,0.00,0.00,0.00,0.00,0.00,nan
5%,0.00,0.00,0.00,0.00,0.00,nan
10%,"10,000.00","5,000.00",0.00,0.00,0.00,nan
25%,"21,000.00","15,000.00",0.00,0.00,0.00,nan
50%,"37,000.00","25,000.00","6,000.00",0.00,0.00,nan
75%,"67,000.00","38,750.00","18,000.00","5,000.00",0.00,nan
80%,"72,000.00","40,400.00","24,000.00","5,000.00",0.00,nan


In [14]:
# =============================================================================
# Data per plot
# =============================================================================
data = data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack()

### I get a problem since a hosuehold was interviewed twice:
data_216 = data.loc[data['hhid']==10, ]  ## SOLVED: remove one entry
data.drop((data.loc[(data['hhid']==216) & (data['duration']==3666)]).index, inplace=True)

# Generate empty dataset
N_p= np.sum(data['total_plots'])
ones = np.ones((int(N_p),2))
data_plots = pd.DataFrame({'hhid':ones[:,0], 'plotid':ones[:,1]})

## Populate dataset with hhid and plotid
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'total_plots'])+1):
        i+=1
        data_plots.iloc[i,0] = hhid
        data_plots.iloc[i,1] = plot
 
       
## generate variables:
# List of chosen crops.. If not chosen then the variables associated to not-chosen crop are unexistent. Update this list
# Everytime we get new data. Check sum_kg for a quick selection.
list_crops_selected = ['maize', 'groundnut', 'sorghum',  'pigeonpeas'] 
# Code also works with all the crops. This is just to avoid empty columns.

for crop in list_crops:
    data_plots[crop+'_kg'] = np.nan
    

data_plots['area'] = np.nan  #area is already converted in acres
data_plots['rentoutplot'] = np.nan
data_plots['valueplot'] = np.nan
data_plots['kg_fertilizer'] = np.nan

#### Loop for plot characteristics
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'total_plots'])+1):
        i+=1
        data_plots.iloc[i, data_plots.columns.get_loc('area')] = float(data.loc[data['hhid']==hhid, 'area_plot_acr_'+str(plot)])
        ## problem: area of rented-in plots. In this case the one with area=0
        data_plots.iloc[i, data_plots.columns.get_loc('rentoutplot')] = float(data.loc[data['hhid']==hhid, 'rentoutplot_'+str(plot)])
        data_plots.iloc[i, data_plots.columns.get_loc('valueplot')] = float(data.loc[data['hhid']==hhid, 'valueplot_'+str(plot)])
        
#### Looop for fertilizer
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'repeatplotsfertilizer_count'])+1):
        i+=1        
        data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'fertilizerplotsselected_'+str(plot)]))),'kg_fertilizer'] = float(data.loc[data['hhid']==hhid, 'plotkgfertilizer_'+str(plot)])
        
 
#### Loop for crop production       
for hhid in data['hhid']:
        for crop in list_crops:
            for plot in range(1,int(data.loc[data['hhid']==hhid, 'repeatplots'+crop+'_count'])+1):
                #print(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,crop+'plotsselected_'+str(plot)]))), crop+'_kg'] = float(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                #data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,crop+'plotsselected_'+str(plot)]))), crop+'_kg'] = i


#### Loop for labor input 
for member in range(1,int(np.max(data['manyhhlaborplot'])+1)):    
    data_plots['months_member_'+str(member)] = np.nan           
    data_plots['weeks_member_'+str(member)] = np.nan             
    data_plots['days_member_'+str(member)] = np.nan 
    data_plots['hours_member_'+str(member)] = np.nan 
    data_plots['hours_member_'+str(member)] = np.nan
for hhid in data['hhid']:
    for member in range(1,int(data.loc[data['hhid']==hhid, 'manyhhlaborplot'])+1):
        for plot in range(1,int(data.loc[data['hhid']==hhid, 'hhlaborperplotrepeat_count_'+str(member)])+1):
                          
            #print(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'months_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'monthshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'weeks_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'weekshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'days_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'dayshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'hours_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'hourshhplot_'+str(member)+'_'+str(plot)])



data_plots['hh_labor_days'] = 0
data_plots['hh_labor_hours'] = 0

for member in range(1,int(np.max(data['manyhhlaborplot'])+1)):           
    data_plots['member_'+str(member)+'_labor_days'] =  (data_plots['months_member_'+str(member)].multiply(data_plots['weeks_member_'+str(member)],axis=0, fill_value=0)).multiply(data_plots['days_member_'+str(member)],axis=0, fill_value=0)  
    data_plots['member_'+str(member)+'_labor_hours'] = data_plots['member_'+str(member)+'_labor_days'].multiply(data_plots['hours_member_'+str(member)],axis=0, fill_value=0)

for member in range(1,int(np.max(data['manyhhlaborplot'])+1)): 
    
    data_plots['hh_labor_days'] += data_plots['member_'+str(member)+'_labor_days'].fillna(0)
    data_plots['hh_labor_hours'] += data_plots['member_'+str(member)+'_labor_hours'].fillna(0)
    
    

print('===============================================================')
print('Distribution Household Labor in hours')
print('===============================================================')
sum_labor_hours = data_plots[['hh_labor_hours', 'member_1_labor_hours', 'member_2_labor_hours', 'member_3_labor_hours']].describe(percentiles=percentiles)
sum_labor_hours

Distribution Household Labor in hours


,hh_labor_hours,member_1_labor_hours,member_2_labor_hours,member_3_labor_hours
count,470.00,390.00,317.00,168.00
mean,751.09,450.25,367.57,213.94
std,673.21,306.33,285.37,236.90
min,0.00,0.00,0.00,0.00
5%,0.00,36.00,13.60,3.35
10%,0.00,83.60,41.40,32.00
25%,240.00,216.00,120.00,80.00
50%,669.00,432.00,320.00,144.00
75%,"1,116.00",661.50,576.00,224.00
80%,"1,225.60",672.00,667.20,295.20


In [15]:
## Systematic check no plot has input investment (fertilizer) with 0 output ================

## Compare reported aggregate quantity vs summing across plots:
data_plots_agg = data_plots.groupby(by='hhid').sum()
data_plots_agg.reset_index(inplace=True)

data_plots_agg = data_plots_agg[['hhid','maize_kg','groundnut_kg','sorghum_kg', 'pigeonpeas_kg', 'kg_fertilizer','hh_labor_hours']]
data_plots_agg.columns = ['hhid','maize_kg','groundnut_kg','sorghum_kg', 'pigeonpeas_kg', 'kg_fertilizer', 'hh_labor_hours']

data[['kg_fertilizer', 'hh_labor_hours']] = data_plots_agg[['kg_fertilizer', 'hh_labor_hours']]
data_kg_check = data[['hhid', 'intervieweename','hh_area_plots','total_kg_maize', 'total_kg_groundnut','total_kg_sorghum', 'total_kg_pigeonpeas', 'fertilizerkg','enumerator' ]]
data_kg_check['hhid'] = data_kg_check['hhid'].astype('float64')

data_kg_check = data_kg_check.merge(data_plots_agg, on='hhid')  


### create difference. report those households with big differences
list_crops_check = ['maize', 'groundnut', 'pigeonpeas']
for crop in list_crops_check:
    data_kg_check['check_diff_'+crop] = data_kg_check['total_kg_'+crop].fillna(0) - data_kg_check[crop+'_kg'].fillna(0)

data_kg_check['check_diff_fertilizer'] = data_kg_check['fertilizerkg'].fillna(0) - data_kg_check['kg_fertilizer'].fillna(0)

data_diff = data_kg_check[['hhid','intervieweename', 'enumerator','check_diff_maize','check_diff_groundnut','check_diff_pigeonpeas','check_diff_fertilizer']]

data_diff.replace([0,0.0], np.nan, inplace=True)
data_diff.dropna(subset=['check_diff_maize','check_diff_groundnut','check_diff_pigeonpeas','check_diff_fertilizer'], axis=0, how='all',inplace=True)

### These are the households to check:
print('')
print('===============================================================')
print('Check: Households that aggregate vs sum(plots) variables do not coincide')
print('===============================================================')
print('with Raul we alredy checked this table. No need to look at it now.')
#data_diff


### Corrections Augustine
# Maize
data.loc[data['hhid']==93, 'maize_kg'] = 700
data.loc[data['hhid']==41, 'maize_kg'] = 300
data.loc[data['hhid']==106, 'total_kg_maize'] = 150
data.loc[data['hhid']==115, 'total_kg_maize'] = 4.5*50
data.loc[data['hhid']==115, 'maize_kg'] = 4.5*50
data.loc[data['hhid']==171, ['total_kg_maize','store_kg_maize']]  = [12*50, 6*50]
data.loc[data['hhid']==182, 'maize_kg'] = 500
data.loc[data['hhid']==246, 'maize_kg'] = 500
# groundnut
data.loc[data['hhid']==171, 'total_kg_groundnut'] = 25
data.loc[data['hhid']==193, 'total_kg_groundnut'] = 150
data.loc[data['hhid']==255, 'groundnut_kg'] = 150
#pigeonpeas
data.loc[data['hhid']==135, 'pigeonpeas_kg'] = 200
# Fertilizer
data.loc[data['hhid']== 15, ['fertilizerkg','kg_fertilizer','buyfertilizerkg']] = 25
data.loc[data['hhid']== 23, ['fertilizerkg','kg_fertilizer','buyfertilizerkg']] = 25
data.loc[data['hhid']== 30, ['fertilizerkg','kg_fertilizer','buyfertilizerkg']] = 50
data.loc[data['hhid']== 268, ['fertilizerkg','kg_fertilizer']] = 45
data.loc[data['hhid']== 268, ['buyfertilizerkg']] = 50
data.loc[data['hhid']== 214, 'fertilizerkg'] = 150 
data.loc[data['hhid']== 31, ['fertilizerkg']] = 100
data.loc[data['hhid']== 36, ['kg_fertilizer']] = 50
data.loc[data['hhid']== 90, ['kg_fertilizer']] = 50
data.loc[data['hhid']== 132, ['fertilizerkg','kg_fertilizer', 'buyfertilizerkg']] = 75
data.loc[data['hhid']== 208, ['kg_fertilizer']] = 50
data.loc[data['hhid']== 903, ['kg_fertilizer']] = 50
data.loc[data['hhid']== 903, 'buyfertilizerkg'] = 0
data.loc[data['hhid']== 264, ['fertilizerkg','kg_fertilizer']] = 100
data.loc[data['hhid']== 143, ['kg_fertilizer', 'buyfertilizerkg']] = 50
data.loc[data['hhid']== 214, ['fertilizerkg', 'buyfertilizerkg']] = 150
data.loc[data['hhid']== 182, ['fertilizerkg','kg_fertilizer', 'buyfertilizerkg']] = 75

## outlier in maize yields
data.loc[data['hhid']==211, ['total_kg_maize','maize_kg']]  = 175
data.loc[data['hhid']==134, ['total_kg_maize','maize_kg']]  = 10*50
data.loc[data['hhid']==134, 'hh_area_plots'] = 1

data.loc[data['hhid']==134, ['total_kg_maize','maize_kg']]  = 21*50
data.loc[data['hhid']==86, 'hh_area_plots']  = 1.5


data.to_csv('income_data.csv')

C:\Users\rodri\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\rodri\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check: Households that aggregate vs sum(plots) variables do not coincide
with Raul we alredy checked this table. No need to look at it now.


In [16]:
#%% =============================================================================
#  Labor input
# =============================================================================

# total labor number persons


# Hired labor 
list_persons = ['men','women','kids']
data['w_men']=np.nan
data['w_women']=np.nan
data['w_kids']=np.nan
data['hired_N'] = 0

for person in list_persons:
    data['hired_N'] += data['manyhired'+str(person)].fillna(0)
    data['hired_'+str(person)+'_avg_hours'] = (data['hireplotmotnhs'+str(person)]*data['hireplotweeks'+str(person)]*data['hireplotdays'+str(person)]*data['hireplothours'+str(person)]).replace(0,np.nan)
    data['hired_'+str(person)+'_L'] = data['manyhired'+str(person)]*data['hired_'+str(person)+'_avg_hours'] 
    data['w_'+str(person)] = (data['hireplotwage'+str(person)].replace(0,np.nan) / data['hired_'+str(person)+'_avg_hours'])
    data['weight_'+str(person)] = np.nanmedian(data['w_'+str(person)])   
 
sum_hiredlabor = data[['w_men','w_women','w_kids' , 'hired_men_avg_hours', 'hired_women_avg_hours','hired_kids_avg_hours' , 'hireplotwagemen', 'hireplotwagewomen', 'hireplotwagekids']].describe()
print('=========== Summary Hired Labor ===============')
sum_hiredlabor   
  

=========== Summary Hired Labor ===============


,w_men,w_women,w_kids,hired_men_avg_hours,hired_women_avg_hours,hired_kids_avg_hours
count,32.00,20.00,30.00,32.00,21.00,32.00
mean,226.37,175.24,225.53,112.16,73.10,43.47
std,228.29,155.37,374.48,128.38,129.89,55.68
min,14.81,13.33,5.56,12.00,2.00,1.00
25%,97.50,60.76,21.88,27.25,14.00,6.00
50%,140.62,108.33,65.08,55.00,24.00,12.00
75%,250.00,270.83,161.46,152.50,72.00,72.00
max,911.46,500.00,"1,500.00",540.00,600.00,216.00


In [21]:
data['hhlabor_N'] = data['manyhhlaborplot']
data['labor_N'] = (data['hhlabor_N'].fillna(0) +data['hired_N'].fillna(0)).replace(0,np.nan)
data['labor_h'] = (data['hh_labor_hours'].fillna(0) +data['hired_men_L'].fillna(0) +data['hired_women_L'].fillna(0) +data['hired_kids_L'].fillna(0)).replace(0,np.nan)


sum_agriclabor = data[['labor_N', 'labor_h', 'hired_N', 'hh_labor_hours','hired_men_L','hired_women_L', 'hired_kids_L']].describe()
print('=========== Summary Household + Hired Agricultural Labor input ===============')
print('Where _N denotes in supply number of persons, _h or _L in total hours')
sum_agriclabor



=========== Summary Household + Hired Agricultural Labor input ===============
Where _N denotes in supply number of persons, _h or _L in total hours


,labor_N,labor_h,hired_N,hh_labor_hours
count,243.00,243.00,255.00,250.00
mean,3.70,"1,582.19",1.14,"1,412.05"
std,3.01,"1,466.64",2.83,"1,222.86"
min,1.00,2.00,0.00,0.00
25%,2.00,719.00,0.00,622.50
50%,3.00,"1,272.00",0.00,"1,156.00"
75%,4.00,"2,016.00",0.00,"1,842.00"
max,22.00,"14,202.00",22.00,"10,824.00"


In [29]:
# obtain value non-bought fertilizer. Use median price
data['p_fert']  = pd.to_numeric(data['buyfertilizierpay'].divide(data['buyfertilizerkg'].replace([0,0.0], np.nan)))

## Use kg of fertilizer by summing across plots
data['value_fertilizer'] = np.nanmedian(data['p_fert'])*data['kg_fertilizer']

#intermediates
data['interm'] = (data['spendseeds'].fillna(0) +data['value_fertilizer'].fillna(0) +data['spendpesticides'].fillna(0)).replace(0,np.nan)

sum_interm = data[['interm','value_fertilizer','kg_fertilizer','spendseeds', 'spendpesticides']].describe(percentiles=percentiles)

print('=========== Summary Intermediate inputs ===============')
print('All variables in MWK except kg_fertilizer.')
sum_interm


=========== Summary Intermediate inputs ===============
All variables in MWK except kg_fertilizer.


,interm,value_fertilizer,kg_fertilizer
count,237.00,251.00,251.00
mean,"13,107.47","9,129.88",45.65
std,"10,168.50","8,989.34",44.95
min,600.00,0.00,0.00
5%,"2,000.00",0.00,0.00
10%,"3,000.00",0.00,0.00
25%,"5,000.00","4,000.00",20.00
50%,"10,500.00","6,000.00",30.00
75%,"18,000.00","10,000.00",50.00
80%,"20,000.00","15,000.00",75.00


In [39]:
### LABOR INCOME: SALARY LABOR (last rainy season) ----------------------

## propotion households getting sallary work:
data['lobor_inc1'].replace(2,0, inplace=True)
pd.value_counts(data['lobor_inc1'])/len(data)

data['wlabor'] = 0
## labor supply in hours
data['wlabor_supply'] = ((data['lobor_inc3'].multiply(data['lobor_inc4'],axis=0, fill_value=0)).multiply(data['lobor_inc5'],axis=0, fill_value=0)).multiply(data['lobor_inc6'],axis=0, fill_value=0)   

## Construct wlabor income for the rainy season
pd.value_counts(data['lobor_inc8'])
data.loc[data['lobor_inc8']==1,'wlabor_inc'] = data.loc[data['lobor_inc8']==1, 'lobor_inc7'].multiply(data['lobor_inc3'],axis=0, fill_value=0)
data.loc[data['lobor_inc8']==2, 'wlabor_inc'] = (data.loc[data['lobor_inc8']==2, 'lobor_inc7'].multiply(data['lobor_inc4'],axis=0, fill_value=0)).multiply(data['lobor_inc3'],axis=0, fill_value=0)

data['wlabor_inc'] = pd.to_numeric(data['wlabor_inc'])
data['wlabor_supply'] = pd.to_numeric(data['wlabor_supply'])

euro_MWK = 806.64
data['wlabor_inc_euro'] = data['wlabor_inc']/euro_MWK

## LABOR INCOME: GANYU (last 3 months) -----------------------------------
data['lobor_inc9'].replace(2,0, inplace=True)
pd.value_counts(data['lobor_inc9'])/len(data)
data['wlabor'] = 0

## labor supply in hours
data['ganyu_supply'] = (data['lobor_inc10'].multiply(data['lobor_inc11'],axis=0, fill_value=0)).multiply(data['lobor_inc12'],axis=0, fill_value=0) 

(data[['lobor_inc10','lobor_inc11','lobor_inc12','ganyu_supply']].replace(0, np.nan)).describe()
## average weeks per 3 months
data['lobor_inc10'].replace(0,np.nan).describe()
## Construct wage per hour
pd.value_counts(data['lobor_inc13_period']) ## most people reported in ganyu per task. followed by weekly and daily
### need to know how many ganyus...
# daily
data.loc[data['lobor_inc13_period']==2, 'ganyu_inc'] = (data.loc[data['lobor_inc13_period']==2, 'lobor_inc13'].multiply(data['lobor_inc11'],axis=0, fill_value=0)).multiply(data['lobor_inc10'],axis=0, fill_value=0) 
#weekly
data.loc[data['lobor_inc13_period']==3, 'ganyu_inc'] = data.loc[data['lobor_inc13_period']==3, 'lobor_inc13'].multiply(data['lobor_inc10'],axis=0, fill_value=0)
#montly
data.loc[data['lobor_inc13_period']==4, 'ganyu_inc'] = data.loc[data['lobor_inc13_period']==4, 'lobor_inc13']*3


##compute median ganyu payment to assign to those who replied for task.
ganyu_25guys = (data[['ganyu_inc', 'ganyu_supply']].replace(0,np.nan)).dropna()
wganyu_hour = np.mean(ganyu_25guys['ganyu_inc']) / np.mean(ganyu_25guys['ganyu_supply'])
data.loc[data['lobor_inc13_period']==5,'ganyu_inc'] = data.loc[data['lobor_inc13_period']==5, 'ganyu_supply']*wganyu_hour  ## I assume 1 ganyu..


## to rainy season:
data['ganyu_inc'] = pd.to_numeric(data['ganyu_inc']*(7/3))
data['ganyu_supply'] = pd.to_numeric(data['ganyu_supply']*(7/3))
data['ganyu_inc_euro'] = pd.to_numeric(data['ganyu_inc']/euro_MWK)


sum_nonagri_labor = data[['wlabor_inc_euro','wlabor_supply','ganyu_inc_euro','ganyu_supply']].describe(percentiles=percentiles)
print('===============================================================')
print('Salary and Ganyu labor income and supply (at rainy season, 7 months)')
print('===============================================================')
print('income in euros')
sum_nonagri_labor

Salary and Ganyu labor income and supply (at rainy season, 7 months)
income in euros


,wlabor_inc_euro,wlabor_supply,ganyu_inc_euro,ganyu_supply
count,35.00,255.00,80.00,255.00
mean,434.11,57.78,783.00,59.41
std,309.93,224.82,"1,106.00",140.84
min,0.00,0.00,0.00,0.00
5%,16.22,0.00,8.68,0.00
10%,50.58,0.00,28.77,0.00
25%,264.80,0.00,86.78,0.00
50%,433.90,0.00,286.89,0.00
75%,642.67,0.00,"1,168.06",42.00
80%,642.67,0.00,"1,366.15",70.93


In [43]:
### BUSINESS INCOME (last month) -------------------------------------------
data['busin_income_1'].replace(2,0)
pd.value_counts(data['busin_income_1'])
type_business = pd.value_counts(data['busin_income_2'])
data['business_type'] = data['busin_income_2']
data['business_revenue'] = data['busin_income_3']
data['business_costs'] = data['busin_income_4'] + data['busin_income_5']
data['business_profits'] = data['business_revenue'] - data['business_costs']

business_data = data.loc[data['busin_income_1']==1, ['hhid', 'business_type', 'business_revenue', 'business_costs', 'business_profits']]
business_data[['business_revenue', 'business_costs', 'business_profits']] = business_data[['business_revenue', 'business_costs', 'business_profits']]/euro_MWK

data['business_revenue'] =  pd.to_numeric(data['business_revenue'])
data['business_costs'] =  pd.to_numeric(data['business_costs'])
data['business_profits'] =  pd.to_numeric(data['business_profits'])

### to rainy season: 
data[['business_revenue', 'business_costs', 'business_profits']] =data[['business_revenue', 'business_costs', 'business_profits']]*7

sum_business = (data[['business_revenue', 'business_costs', 'business_profits']].replace(0,np.nan)/euro_MWK).describe(percentiles=percentiles)
print('===============================================================')
print('Summary Business income at rainy season')
print('===============================================================')
print('values in euros')
sum_business


Summary Business income at rainy season
values in euros


,business_revenue,business_costs,business_profits
count,44.00,25.00,44.00
mean,345.06,247.43,204.47
std,470.39,503.24,291.06
min,17.36,4.34,-199.59
5%,43.39,8.68,-72.89
10%,45.99,9.09,16.14
25%,82.44,30.37,43.39
50%,151.86,52.07,95.46
75%,433.90,173.56,282.03
80%,468.61,194.39,400.92


In [45]:

### OTHER SOURCES OF INCOME --------------------------------------------------

data['cashtrans_yes'] = data['other_sour_income_3'].replace(2,0)
data['cashtrans_value'] = data['other_sour_income_4'].replace(0,np.nan)
data[['NGO_yes', 'cashtrans_yes', 'gov_yes', 'remittances_yes']] = data[['other_sour_income_1', 'other_sour_income_3', 'other_sour_income_5', 'other_sour_income_7']]
data[['NGO_yes', 'cashtrans_yes', 'gov_yes', 'remittances_yes']] = data[['NGO_yes', 'cashtrans_yes', 'gov_yes', 'remittances_yes']].replace(2,0)
sum_other_prop = np.mean(data[['NGO_yes', 'cashtrans_yes', 'gov_yes', 'remittances_yes']], axis=0)

data['NGO_trans'] = data['other_sour_income_2']
data['gov_trans'] = data['other_sour_income_6']
data['remittances'] = data['other_sour_income_8']
data['other_inc'] = data[['cashtrans_value', 'NGO_trans', 'gov_trans', 'remittances']].sum(axis=1)
sum_other = (data[['cashtrans_value', 'NGO_trans', 'gov_trans', 'remittances']].replace(0,np.nan)/euro_MWK).describe()


### AGGREGATE INCOME ------------------------------------------------------------
data['inctotal'] = data[['y_agric' ,'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']].sum(axis=1)
income = data[['hhid','inctotal','y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']].replace(0,np.nan)

sum_inc = (income.loc[:, income.columns != 'hhid']/euro_MWK).describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

var_list = ['inctotal','y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']
gini_stat= np.empty((1, len(var_list)))

for i,state in enumerate(var_list):
    gini_stat[:,i] = gini(income[state].dropna().values)

data_gini = pd.DataFrame(gini_stat, columns=var_list)    
data_gini.reset_index(inplace=True)
data_gini['index'] = 'gini'
sum_inc.reset_index(inplace=True)
sum_inc = sum_inc.append(data_gini, ignore_index=True)

print('===============================================================')
print('Summary total Income (at rainy season)')
print('===============================================================')
print('values in MWK')
sum_inc


Summary total Income (at rainy season)
values in MWK


,index,inctotal,y_agric,y_maize,y_groundnut,y_pigeonpeas,wlabor_inc,ganyu_inc,business_profits,other_inc
0,count,249.00,240.00,237.00,156.00,103.00,34.00,78.00,44.00,157.00
1,mean,446.06,64.77,41.78,24.84,11.23,446.88,803.07,204.47,55.38
2,std,753.71,71.06,52.71,21.52,10.76,305.10,"1,112.96",291.06,85.78
3,min,-137.61,0.62,0.62,0.74,0.25,12.40,4.34,-199.59,2.17
4,10%,34.71,17.66,12.40,3.72,1.49,76.61,32.98,16.14,5.95
5,25%,68.18,29.75,18.60,7.44,4.96,275.84,92.41,43.39,11.16
6,50%,143.81,49.59,30.99,22.31,6.20,433.90,295.31,95.46,23.55
7,75%,506.42,85.54,49.59,37.19,12.40,642.67,"1,209.04",282.03,70.66
8,90%,"1,094.07",120.54,69.42,52.07,24.79,837.25,"2,065.62",464.27,124.96
9,max,"5,314.20",883.91,694.24,111.57,74.38,"1,487.65","5,246.01","1,388.48",743.83
